In [5]:
%%writefile /dbfs/lib/dbutils.py

import psycopg2, logging
import psycopg2.extras
import pandas as pd

def db_query(conn_params, query, query_params, my_parser, logger=None):
    [conn, cur] = execute_query(conn_params, query, query_params, logger)
    df = my_parser(cur)
    cur.close()
    conn.close()
    return df


def db_query_as_df(conn_params, query, query_params, logger=None):
    [conn, cur] = execute_query(conn_params, query, query_params, logger)
    df = pd.DataFrame(cur.fetchall())
    df.columns = [desc[0] for desc in cur.description]
    cur.close()
    conn.close()
    return df


def execute_query(conn_params, query, query_params, logger=None):
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor()
    if logger is not None:
        logger.log(logging.DEBUG, "query=" + query + ",query_params=" + str(query_params))
    cur.execute(query, query_params)
    return [conn, cur]


def db_insert(conn_params, sql, values):
    import psycopg2.extras
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor()
    psycopg2.extras.execute_values(cur, sql, values, template=None, page_size=500)
    conn.commit()

Overwriting /dbfs/lib/dbutils.py
